In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import lsst.daf.butler as dB
import lsst.cp.verify.notebooks.utils as utils
import lsst.afw.display as afwDisplay

In [ ]:
# This cell contains parameters that can be automatically set via the papermill package.
# Examples:
#   papermill --prepare-only -p calibType newBias -p cameraName LSSTCam <input> <output>
#   papermill -p interactive False <input> <output>
interactive = True

# Which repository to use.
repository = '/repo/main/'

# Which calibration type to analyse.
calibType = 'cpPtcExtract'
exposureNumber = 2022021800078

# Which camera the calibration is for.
cameraName = 'LSSTComCam'

# Which display to use.
displayBackend = 'matplotlib'

# Which collection the calibration was constructed in.
genCollection = 'u/plazas/2022SEP20-ptc.comcam.DM-34867-FULLCOV.1'

# Which collection containing the verification outputs.
verifyCollection = 'u/plazas/DM-34867-2022SEP26-VERIFY-gain.comcam.FULLCOV.5'

In [ ]:
# Get butler and camera
# the genCollection has the camera
butler = dB.Butler(repository, collections=[genCollection,verifyCollection])
camera = butler.get('camera', instrument=cameraName)
numberDetectors = len(camera)

In [ ]:
# Get Run Statistics
runStats = butler.get('verifyGainStats', instrument=cameraName)
runSuccess = runStats.pop('SUCCESS')

In [ ]:
# Display summary table of tests and failure counts.
utils.failureTable(runStats)

In [ ]:
# Let's look at the values to understand better the table from above.
# calibType with gain and ISR noise is `cpPtcExtract`, from the gain-from-flat-pairs 
# subset in the cpPtc.yaml
gainThreshold = 5  # percentage, default value in config
noiseThreshold = 5 # percentage, default value in config
for detector in camera:
    detId = detector.getId()
    calib = butler.get(calibType, instrument=cameraName, 
                       detector=detId, exposure=exposureNumber)
    print (f"Detector ID: {detId}")
    for amp in detector:
        ampName = amp.getName()
        print (f"    {ampName}")
        diffGain = 100* (np.abs(calib.gain[ampName] - amp.getGain()) / amp.getGain())
        diffNoise = 100* (np.abs(calib.noise[ampName] - amp.getReadNoise()) / amp.getReadNoise())
        testGain = bool(diffGain > gainThreshold)
        testNoise = bool(diffNoise > noiseThreshold)
        print ( f"Noise --  Difference(%): {diffNoise}, ISR noise: {calib.noise[ampName]}, Amp. Noise {amp.getReadNoise()}")
        print ( f"Gain  --  Difference(%): {diffGain}, Gain from pair: {calib.gain[ampName]}, Amp. gain {amp.getGain()}")
        print (" ")
    print(" ")
    print(" ")

In [ ]:
# Gain, ISR noise and other values from 'cpPtcExtract' are saved as 'AMP' statistics
for detector in camera:
    detId = detector.getId()
    detStats = butler.get("verifyGainDetStats", instrument=cameraName, detector=detId)
    print ("Detector ID: ", detId) 
    print ("    Overall success: ", detStats['SUCCESS'])
    print ("    Overall success per test type: ", detStats['VERIFY'])
    for ampName in detStats['AMP']:
        print (f"    {ampName}", detStats['AMP'][ampName])
    print (" ")